## DLthon 텍스트데이터 분류

- 협박, 갈취, 직장 내 괴롭힘, 기타 괴롭힘, 그리고 일반의 총 5가지 대화 유형 클래스를 분류하는 모델

In [112]:
import pandas as pd
import re


### 합성데이터 합치기

In [21]:
# 데이터 파일 경로 리스트
file_paths = [
    "data/train.csv",
    "data/BJSon_result_conversations.csv",
    "data/CHYeom_general_conversations.csv",
    "data/kimcj_all_conversations.csv",
    "data/YEKim_general_conversations.csv"
]

# 데이터프레임 리스트 생성 및 크기 출력
dfs = []
for i, path in enumerate(file_paths, start=1):
    try:
        df = pd.read_csv(path, sep=None, engine='python', on_bad_lines='skip')  # 여러 문제 해결
        dfs.append(df)
        print(f"df{i} 크기 : {len(df)}")
    except Exception as e:
        print(f"❌ {path} 파일을 읽는 중 오류 발생: {e}")

# 데이터 합치기
merged_df = pd.concat(dfs, ignore_index=True)

# 불필요한 칼럼 제거
merged_df = merged_df.drop(columns=['﻿idx', 'index'], errors='ignore')

# 'idx' 칼럼 인덱스값 업데이트
merged_df['idx'] = merged_df.index

# 칼럼 순서 조정
merged_df = merged_df[['idx', 'class', 'conversation']]

# 저장
merged_df.to_csv("data/merged_data.csv", index=False)

# 결과 확인
print("merged_df 크기 : ", len(merged_df))
print(merged_df.head())


df1 크기 :  3950
df2 크기 :  305
merged_df 크기 :  4255
   idx      class                                       conversation
0    0      협박 대화  지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1    1      협박 대화  길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2    2  기타 괴롭힘 대화  너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3    3      갈취 대화  어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4    4      갈취 대화  저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...


### 데이터 전처리

In [144]:
train_df = pd.read_csv("/content/drive/MyDrive/data/train_ch.csv")
test_df = pd.read_csv("/content/drive/MyDrive/data/test.csv")

In [145]:
len(train_df), len(test_df)

(4788, 500)

In [146]:
train_df.head()

,Unnamed: 0,idx,class,conversation
0,0,1,0,지금 너 스스로를 죽여달라고 애원하는 것인가 ? 아닙니다 . 죄송합니다 . 죽을 거...
1,1,2,0,길동경찰서입니다 . 시 분 마트에 폭발물을 설치할거다 . 네 ? 똑바로 들어 ...
2,2,3,3,너 되게 귀여운거 알지 ? 나보다 작은 남자는 첨봤어 . 그만해 . 니들 놀리는거 ...
3,3,4,1,어이 거기 예 ? ? 너 말이야 너 . 이리 오라고 무슨 일 . 너 옷 좋아보인다 ...
4,4,5,1,저기요 혹시 날이 너무 뜨겁잖아요 ? 저희 회사에서 이 선크림 파는데 한 번 손등에...


In [147]:
train_df.drop(columns=['Unnamed: 0'], inplace=True)

In [148]:
test_df.head()

,idx,text
0,t_000,아가씨 담배한갑주소 네 4500원입니다 어 네 지갑어디갔지 에이 버스에서 잃어버렸나...
1,t_001,우리팀에서 다른팀으로 갈 사람 없나? 그럼 영지씨가 가는건 어때? 네? 제가요? ...
2,t_002,너 오늘 그게 뭐야 네 제가 뭘 잘못했나요.? 제대로 좀 하지 네 똑바로 좀 하지 ...
3,t_003,이거 들어바 와 이 노래 진짜 좋다 그치 요즘 이 것만 들어 진짜 너무 좋다 내가 ...
4,t_004,아무튼 앞으로 니가 내 와이파이야. .응 와이파이 온. 켰어. 반말? 주인님이라고도...


In [149]:
# 결측치 확인
train_df.isnull().sum()

,0
idx,0
class,0
conversation,0


In [150]:
# 중복 데이터 확인
train_df.duplicated().sum()

0

#### 텍스트 정제

In [151]:
def preprocess_sentence(sentence):
    # 양쪽 공백을 제거
    sentence = sentence.strip()

    # 줄바꿈(\n)을 공백으로 변환
    sentence = sentence.replace("\n", " ")

    # 단어와 구두점(punctuation) 사이의 거리를 만듭니다.
    # 예를 들어서 "I am a student." => "I am a student ."와 같이
    # student와 온점 사이에 거리를 만듭니다.
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)

    # (가-힣)를 제외한 모든 문자를 공백인 ' '로 대체합니다.
    # ".", "?", "!", "," 삭제
    sentence = re.sub(r"[^가-힣]", " ", sentence)
    sentence = sentence.strip()
    return sentence

train_df['conversation'] = train_df['conversation'].apply(preprocess_sentence)

In [152]:
train_df.head()

,idx,class,conversation
0,1,0,지금 너 스스로를 죽여달라고 애원하는 것인가 아닙니다 죄송합니다 죽을 거...
1,2,0,길동경찰서입니다 시 분 마트에 폭발물을 설치할거다 네 똑바로 들어 한번만...
2,3,3,너 되게 귀여운거 알지 나보다 작은 남자는 첨봤어 그만해 니들 놀리는거 ...
3,4,1,어이 거기 예 너 말이야 너 이리 오라고 무슨 일 너 옷 좋아보인다 ...
4,5,1,저기요 혹시 날이 너무 뜨겁잖아요 저희 회사에서 이 선크림 파는데 한 번 손등에...


#### 라벨인코딩

In [121]:
train_df["class"].unique()

array([0, 3, 1, 2, 4])

In [123]:
label_mapping = {
    "협박 대화": 0,
    "갈취 대화": 1,
    "직장 내 괴롭힘 대화": 2,
    "기타 괴롭힘 대화": 3,
    "일반": 4
}

In [124]:
# 매핑 적용
train_df['class'] = train_df['class'].map(label_mapping)

train_df.head()

,Unnamed: 0,idx,class,conversation
0,0,1,NaN,지금 너 스스로를 죽여달라고 애원하는 것인가 아닙니다 죄송합니다 죽을 거...
1,1,2,NaN,길동경찰서입니다 시 분 마트에 폭발물을 설치할거다 네 똑바로 들어 한번만...
2,2,3,NaN,너 되게 귀여운거 알지 나보다 작은 남자는 첨봤어 그만해 니들 놀리는거 ...
3,3,4,NaN,어이 거기 예 너 말이야 너 이리 오라고 무슨 일 너 옷 좋아보인다 ...
4,4,5,NaN,저기요 혹시 날이 너무 뜨겁잖아요 저희 회사에서 이 선크림 파는데 한 번 손등에...


### koElectra 모델

#### 토큰나이저

In [153]:
from transformers import ElectraTokenizer

tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-small-v3-discriminator")


In [154]:
tokenizer

ElectraTokenizer(name_or_path='monologg/koelectra-small-v3-discriminator', vocab_size=35000, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [155]:
import numpy as np

# 문장별 토큰 개수 계산
lengths = train_df["conversation"].apply(lambda x: len(tokenizer.tokenize(x)))

# 평균, 최대, 90% 백분위수 확인
print(f"평균 길이: {np.mean(lengths):.2f}")
print(f"최대 길이: {np.max(lengths)}")
print(f"90% 백분위수: {np.percentile(lengths, 90)}")
print(f"95% 백분위수: {np.percentile(lengths, 95)}")
print(f"99% 백분위수: {np.percentile(lengths, 99)}")


평균 길이: 101.63
최대 길이: 431
90% 백분위수: 163.0
95% 백분위수: 197.0
99% 백분위수: 290.1300000000001


In [156]:
MAX_LENGTH = 200

In [157]:
from sklearn.model_selection import train_test_split
'''
# 1차 분할: train(80%) / temp(20%)
train_texts, temp_texts, train_labels, temp_labels = train_test_split(
    train_df["conversation"], train_df["class"], test_size=0.2, random_state=42)

# 2차 분할: val(10%) / test(10%)
val_texts, test_texts, val_labels, test_labels = train_test_split(
    temp_texts, temp_labels, test_size=0.5, random_state=42)

print(f"Train 데이터 개수: {len(train_texts)}")
print(f"Validation 데이터 개수: {len(val_texts)}")
print(f"Test 데이터 개수: {len(test_texts)}")
'''

# 1차 분할: train(80%) / temp(20%)
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_df["conversation"], train_df["class"], test_size=0.2, random_state=42)

print(f"Train 데이터 개수: {len(train_texts)}")
print(f"Validation 데이터 개수: {len(val_texts)}")


Train 데이터 개수: 3830
Validation 데이터 개수: 958


In [158]:
# 토큰화 함수 정의
def tokenize_function(texts):
    return tokenizer(
        texts.tolist(),  # 리스트 형태로 변환
        truncation=True,
        padding="max_length",
        max_length=200,
        return_tensors="tf",
        return_attention_mask=True
    )

# Train 데이터 토큰화
train_encodings = tokenize_function(train_texts)
train_input_ids = train_encodings["input_ids"]
train_attention_masks = train_encodings["attention_mask"]

# Validation 데이터 토큰화
val_encodings = tokenize_function(val_texts)
val_input_ids = val_encodings["input_ids"]
val_attention_masks = val_encodings["attention_mask"]

# # Test 데이터 토큰화
# test_encodings = tokenize_function(test_texts)
# test_input_ids = test_encodings["input_ids"]
# test_attention_masks = test_encodings["attention_mask"]

print(f"Train input shape: {train_input_ids.shape}")
print(f"Validation input shape: {val_input_ids.shape}")
# print(f"Test input shape: {test_input_ids.shape}")


Train input shape: (3830, 200)
Validation input shape: (958, 200)


In [159]:
import tensorflow as tf

# TensorFlow Dataset 변환 함수
def encode_tf_dataset(input_ids, attention_masks, labels):
    return tf.data.Dataset.from_tensor_slices((
        {"input_ids": input_ids, "attention_mask": attention_masks},  # 모델 입력
        tf.convert_to_tensor(labels, dtype=tf.int32)  # 정답 라벨
    ))

# Train 데이터셋 변환
train_dataset = encode_tf_dataset(train_input_ids, train_attention_masks, train_labels)

# Validation 데이터셋 변환
val_dataset = encode_tf_dataset(val_input_ids, val_attention_masks, val_labels)

# Test 데이터셋 변환
# test_dataset = encode_tf_dataset(test_input_ids, test_attention_masks, test_labels)

# 배치 크기 설정
BATCH_SIZE = 64
train_dataset = train_dataset.shuffle(len(train_input_ids)).batch(BATCH_SIZE)
val_dataset = val_dataset.batch(BATCH_SIZE)
# test_dataset = test_dataset.batch(BATCH_SIZE)

print("🚀 TensorFlow Dataset 변환 완료!")


🚀 TensorFlow Dataset 변환 완료!


In [143]:
# from transformers import TFElectraForSequenceClassification

# # KoElectra 모델 로드 (PyTorch 모델을 변환하여 사용)
# model = TFElectraForSequenceClassification.from_pretrained(
#     "monologg/koelectra-small-v3-discriminator",
#     num_labels=5,
#     from_pt=True  # PyTorch 모델을 TensorFlow로 변환
# )

# # ✅ 옵티마이저를 `legacy.Adam`으로 변경하여 호환성 문제 해결
# optimizer = tf.optimizers.Adam(learning_rate=2e-5)

# loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# metrics = ["accuracy"]

# # 모델 컴파일
# model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

# # 학습 시작
# EPOCHS = 20
# history = model.fit(
#     train_dataset,
#     validation_data=val_dataset,
#     epochs=EPOCHS
# )

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFElectraForSequenceClassification: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight', 'electra.embeddings.position_ids', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing TFElectraForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFElectraForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFElectraForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dens

Epoch 1/20
60/60 [==============================] - 86s 590ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 2/20
60/60 [==============================] - 31s 525ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 3/20
60/60 [==============================] - 32s 530ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 4/20
60/60 [==============================] - 32s 529ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 5/20
 8/60 [===>..........................] - ETA: 24s - loss: nan - accuracy: 0.0000e+00

KeyboardInterrupt: 

In [160]:
import os
import numpy as np
from transformers import TFElectraForSequenceClassification
import tensorflow as tf

# ✅ 저장할 디렉토리 설정
checkpoint_dir = "./koelectra_best_model"
os.makedirs(checkpoint_dir, exist_ok=True)

# ✅ KoElectra 모델 로드
model = TFElectraForSequenceClassification.from_pretrained(
    "monologg/koelectra-small-v3-discriminator",
    num_labels=5,
    from_pt=True  # PyTorch → TensorFlow 변환
)

# ✅ 옵티마이저 고정 (최신 옵티마이저 사용 시 에러 방지)
optimizer = tf.optimizers.Adam(learning_rate=2e-5)

# ✅ 손실 함수 및 메트릭 설정
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = ["accuracy"]

# ✅ 모델 컴파일
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

# ✅ 학습 시작 (EarlyStopping + 최적의 모델 저장)
EPOCHS = 100
patience = 5  # EarlyStopping 조건 (5번 연속 개선되지 않으면 중단)
wait = 0  # EarlyStopping을 위한 카운터
best_val_accuracy = 0.0
best_epoch = 0

for epoch in range(EPOCHS):
    print(f"\n🔹 Epoch {epoch + 1}/{EPOCHS} 시작...")
    history = model.fit(train_dataset, validation_data=val_dataset, epochs=1, verbose=1)

    # ✅ 현재 epoch의 검증 정확도 가져오기
    current_val_accuracy = history.history["val_accuracy"][0]

    # ✅ 최적의 모델 저장 (가장 높은 val_accuracy를 갖는 모델만 저장)
    if current_val_accuracy > best_val_accuracy:
        best_val_accuracy = current_val_accuracy
        best_epoch = epoch + 1
        wait = 0  # EarlyStopping 카운터 초기화
        model.save_weights(os.path.join(checkpoint_dir, "best_model.weights.h5"))
        print(f"✅ 새로운 최고 성능! Epoch {best_epoch}, val_accuracy={best_val_accuracy:.4f} → 가중치 저장 완료!")
    else:
        wait += 1
        print(f"⚠️ {wait}/{patience} - 검증 정확도 개선 없음.")

    # ✅ EarlyStopping 조건 충족 시 학습 중단
    if wait >= patience:
        print(f"\n⏹️ EarlyStopping 발동! {patience}번 연속 개선되지 않음 → 학습 종료")
        break

print(f"\n🔥 최적의 모델은 Epoch {best_epoch}에서 val_accuracy={best_val_accuracy:.4f}로 저장됨!")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFElectraForSequenceClassification: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight', 'electra.embeddings.position_ids', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing TFElectraForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFElectraForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFElectraForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dens


🔹 Epoch 1/100 시작...
60/60 [==============================] - 98s 627ms/step - loss: 1.5939 - accuracy: 0.3407 - val_loss: 1.5368 - val_accuracy: 0.5251


/usr/local/lib/python3.11/dist-packages/transformers/generation/tf_utils.py:465: UserWarning: `seed_generator` is deprecated and will be removed in a future version.
  warnings.warn("`seed_generator` is deprecated and will be removed in a future version.", UserWarning)


✅ 새로운 최고 성능! Epoch 1, val_accuracy=0.5251 → 가중치 저장 완료!

🔹 Epoch 2/100 시작...
60/60 [==============================] - 36s 608ms/step - loss: 1.4667 - accuracy: 0.5193 - val_loss: 1.3667 - val_accuracy: 0.6023
✅ 새로운 최고 성능! Epoch 2, val_accuracy=0.6023 → 가중치 저장 완료!

🔹 Epoch 3/100 시작...
60/60 [==============================] - 35s 586ms/step - loss: 1.3165 - accuracy: 0.6389 - val_loss: 1.2106 - val_accuracy: 0.6879
✅ 새로운 최고 성능! Epoch 3, val_accuracy=0.6879 → 가중치 저장 완료!

🔹 Epoch 4/100 시작...
60/60 [==============================] - 36s 604ms/step - loss: 1.1590 - accuracy: 0.7371 - val_loss: 1.0267 - val_accuracy: 0.8017
✅ 새로운 최고 성능! Epoch 4, val_accuracy=0.8017 → 가중치 저장 완료!

🔹 Epoch 5/100 시작...
60/60 [==============================] - 34s 574ms/step - loss: 0.9810 - accuracy: 0.8209 - val_loss: 0.8467 - val_accuracy: 0.8518
✅ 새로운 최고 성능! Epoch 5, val_accuracy=0.8518 → 가중치 저장 완료!

🔹 Epoch 6/100 시작...
60/60 [==============================] - 34s 567ms/step - loss: 0.8096 - accuracy: 0.8663 - 

In [102]:
# # 최종 성능 평가
# results = model.evaluate(test_dataset)
# print(f"테스트 정확도: {results[1] * 100:.2f}%")

8/8 [==============================] - 1s 173ms/step - loss: 0.3975 - accuracy: 0.9036
테스트 정확도: 90.36%


In [ ]:
# import os

# # 저장할 디렉토리 설정
# checkpoint_dir = "./koelectra_checkpoints"
# os.makedirs(checkpoint_dir, exist_ok=True)

# # 모델 저장 (Hugging Face 방식)
# model.save_pretrained(checkpoint_dir)
# tokenizer.save_pretrained(checkpoint_dir)

# print(f"✅ 모델과 토크나이저가 {checkpoint_dir}에 저장됨.")


In [161]:
# ✅ 저장된 최적 모델 가중치 불러오기
model.load_weights(os.path.join(checkpoint_dir, "best_model.weights.h5"))

print("✅ 최적의 모델 가중치를 불러왔습니다!")


✅ 최적의 모델 가중치를 불러왔습니다!


In [162]:
test_df

,idx,text
0,t_000,아가씨 담배한갑주소 네 4500원입니다 어 네 지갑어디갔지 에이 버스에서 잃어버렸나...
1,t_001,우리팀에서 다른팀으로 갈 사람 없나? 그럼 영지씨가 가는건 어때? 네? 제가요? ...
2,t_002,너 오늘 그게 뭐야 네 제가 뭘 잘못했나요.? 제대로 좀 하지 네 똑바로 좀 하지 ...
3,t_003,이거 들어바 와 이 노래 진짜 좋다 그치 요즘 이 것만 들어 진짜 너무 좋다 내가 ...
4,t_004,아무튼 앞으로 니가 내 와이파이야. .응 와이파이 온. 켰어. 반말? 주인님이라고도...
...,...,...
495,t_495,미나씨 휴가 결제 올리기 전에 저랑 상의하라고 말한거 기억해요? 네 합니다. 보고서...
496,t_496,교수님 제 논문에 제 이름이 없나요? 아 무슨 논문말이야? 지난 번 냈던 논문이...
497,t_497,야 너 네 저요? 그래 너 왜요 돈좀 줘봐 돈 없어요 돈이 왜 없어 지갑은 폼이...
498,t_498,야 너 빨리 안 뛰어와? 너 이 환자 제대로 봤어 안 봤어 어제 저녁부터 계속 보다...


In [165]:
test_df['text'] = test_df['text'].apply(preprocess_sentence)
test_df

,idx,text
0,t_000,아가씨 담배한갑주소 네 원입니다 어 네 지갑어디갔지 에이 버스에서 잃어버렸나...
1,t_001,우리팀에서 다른팀으로 갈 사람 없나 그럼 영지씨가 가는건 어때 네 제가요...
2,t_002,너 오늘 그게 뭐야 네 제가 뭘 잘못했나요 제대로 좀 하지 네 똑바로 좀 하...
3,t_003,이거 들어바 와 이 노래 진짜 좋다 그치 요즘 이 것만 들어 진짜 너무 좋다 내가 ...
4,t_004,아무튼 앞으로 니가 내 와이파이야 응 와이파이 온 켰어 반말 주인...
...,...,...
495,t_495,미나씨 휴가 결제 올리기 전에 저랑 상의하라고 말한거 기억해요 네 합니다 보...
496,t_496,교수님 제 논문에 제 이름이 없나요 아 무슨 논문말이야 지난 번 냈던 논문이...
497,t_497,야 너 네 저요 그래 너 왜요 돈좀 줘봐 돈 없어요 돈이 왜 없어 지갑은 폼이니...
498,t_498,야 너 빨리 안 뛰어와 너 이 환자 제대로 봤어 안 봤어 어제 저녁부터 계속 보...


In [166]:
submission_df = pd.read_csv("/content/drive/MyDrive/data/submission.csv")
submission_df.head()

,file_name,class
0,t_000,NaN
1,t_001,NaN
2,t_002,NaN
3,t_003,NaN
4,t_004,NaN


### 추론

In [ ]:
# from transformers import TFElectraForSequenceClassification, AutoTokenizer

# # 저장된 모델 및 토크나이저 불러오기
# model = TFElectraForSequenceClassification.from_pretrained(checkpoint_dir)
# tokenizer = AutoTokenizer.from_pretrained(checkpoint_dir)

# print("✅ 저장된 모델과 토크나이저를 성공적으로 불러왔음!")


In [167]:
import tensorflow as tf
import pandas as pd

def predict_texts(df):
    predicted_labels = []
    predicted_probs = []

    for text in df["text"]:
        # 입력 데이터 토큰화
        inputs = tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=200,
            return_tensors="tf"
        )

        # 모델 추론 (Softmax 적용 전 logits 출력)
        logits = model(inputs["input_ids"], attention_mask=inputs["attention_mask"]).logits

        # Softmax 적용하여 확률값 계산
        probabilities = tf.nn.softmax(logits, axis=-1)

        # 가장 확률 높은 라벨 예측
        predicted_label = tf.argmax(probabilities, axis=-1).numpy()[0]
        predicted_labels.append(predicted_label)
        predicted_probs.append(probabilities.numpy())

    return predicted_labels, predicted_probs

# 예측 실행
predicted_labels, predicted_probs = predict_texts(test_df)



In [172]:
# `file_name`과 `class` 컬럼을 포함한 DataFrame 생성
submission_df = test_df[["idx"]].copy()
submission_df.rename(columns={"idx": "file_name"}, inplace=True)  # 컬럼명 변경
submission_df["class"] = predicted_labels  # 예측된 숫자 라벨 추가

# 결과 확인
print(submission_df.head())

# CSV 저장
submission_df.to_csv("submission.csv", index=False, encoding="utf-8-sig")
print("✅ submission.csv 저장 완료!")

  file_name  class
0     t_000      1
1     t_001      2
2     t_002      2
3     t_003      4
4     t_004      3
✅ submission.csv 저장 완료!


In [173]:
submission_df

,file_name,class
0,t_000,1
1,t_001,2
2,t_002,2
3,t_003,4
4,t_004,3
...,...,...
495,t_495,2
496,t_496,2
497,t_497,1
498,t_498,2
